In [1]:
import pandas as pd
import requests

In [2]:
url = 'https://www.letour.fr/en/rankings/stage-19'
crypto_url = requests.get(url)
crypto_url

<Response [200]>

In [3]:
body = crypto_url.text

In [7]:
crypto_data = pd.read_html(body)
print(type(crypto_data))
print(len(crypto_data))

ImportError: lxml not found, please install it